In [1]:
%help

# IParaView Kernel

Custom IPython kernel for an easy interface with ParaView.

## Magic commands:

  - `%help` : get this help
  - `%paraview` : open a Qt client connecting to the current kernel server. Return the client ID.
  - `%sync <int>` : synchronize the Notebook camera with the given client ID.
If no id is given then all Qt clients will be synchronized with the jupyter client.
  - `%interact` : display the active render view as an interactive widget in the notebook
  - Every others magic command available from your usual ipython interpreter.

## Accessible variables:

If you want to have full access to the underlying objects, see namespace `__pv`.
Here's some usefull attributes of the namespace :

  - `connection` : holds the connection information of the kernel to the ParaView server
  - `default_view` : a default render view created for ease-of-use sake.
  - `layout` : a default view layout created for ease-of-use sake.
  - `trace` : the last recorded trace from a qt client.

**WARNING**: do not touch these variables unless you know what you're doing

## Using the generated Python trace from the GUI

By default the GUI cannot be used, as only one client (either the python client or the GUi one) can control the server.
To do some action in the GUI, click the red circle button (record button) in the toolbar.
Once you are done, click the record button (now green) again.
A python editor will pop up in the Qt client, showing you the trace of what you've done.
The same trace is accessible directly from the variable `__pv.trace` in your jupyter environment.
It is possible to use this variable in some interesting way through the builtin magic commands.
Here's 2 possible usage :

 - `%pycat __pv.trace` : display the trace through a syntax-highlighted pager.
 - `%load __pv.trace` : load the trace into the current cell


## Initalize Paraview and read data 

In [2]:
%paraview

2

Next we use the XDMFReader to import the simulation data and then create a slice through the curvilinear dataset of the magnetic field

In [3]:
fDir = "/home/jovyan/group_shares/jupyter_research_pokeball_student/hdf5_cheyenne_1"
fIn = fDir + "/" + "sim.000000.xmf"
Read = XDMFReader(FileNames=fIn)

In [4]:
slice1 = Slice(registrationName='Slice1', Input=Read)
slice1.SliceType.Origin = [0.0, 0.0, 0.0]
slice1.SliceType.Normal = [0.0, 0.0, 1.0]

In [5]:
renderView1 = GetActiveViewOrCreate('RenderView')
slice1Display = Show(slice1, renderView1, 'GeometryRepresentation')
ColorBy(slice1Display, ('CELLS', 'Bz'))
colorMap = GetColorTransferFunction('Bz')
colorMap.RescaleTransferFunction(-50,50)

True

In [6]:
renderView1.CameraPosition = [0,-1000, 500]
renderView1.CameraFocalPoint = [-25.0, 0.0, 0.0]
renderView1.CameraParallelScale = 25


Next we compute the magnetic field vector and then trace some field lines starting from a set of seed points generate across the polar cap using the sphere object.

In [7]:
#Create 3D B vector
calc1 = Calculator(registrationName='B', Input=Read)
calc1.Function = 'iHat*Bx+jHat*By+kHat*Bz'
calc1.ResultArrayName = 'B'

In [8]:
#Create sphere for tracing seeds
sphere1 = Sphere(registrationName='Seeds')
sphere1.Radius = 2.25
sphere1.ThetaResolution = 2
sphere1.EndTheta = 90.0
sphere1.PhiResolution = 2
sphere1.StartPhi = 10.0
sphere1.EndPhi = 30.0

streamTracerWithCustomSource1 = StreamTracerWithCustomSource(registrationName='B-lines', Input=calc1, SeedSource=sphere1)
streamTracerWithCustomSource1.Vectors = ['POINTS', 'B']
streamTracerWithCustomSource1.MaximumStreamlineLength = 300.
streamTracerWithCustomSource1.IntegrationDirection = 'BACKWARD'

streamTracerWithCustomSource1Display = Show(streamTracerWithCustomSource1, renderView1, 'GeometryRepresentation')
ColorBy(streamTracerWithCustomSource1Display, ('CELLS', 'B'))

(   7.008s) [paraview        ]vtkSMPVRepresentationPr:288   WARN| vtkSMPVRepresentationProxy (0x55d9f2e00370): Could not determine array range.


Finally we add a sphere to represent the Earth.

In [9]:
Earth = Sphere()
Earth.Radius = 1.0

## Interactive 3D Visualization

Now that we have the objects created and added to a view we can use the interactive widget to manulipate the view further

In [10]:
Show()
Render()
%interact

ViewInteractiveWidget(height=600, layout=Layout(height='auto', width='100%'), width=800)